# Homework 5: Machine Reading

## RNN Based Model

In [1]:
! git clone https://github.com/williamFalcon/squad.git

Cloning into 'squad'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 47 (delta 5), reused 3 (delta 0), pack-reused 33
Unpacking objects: 100% (47/47), done.


In [2]:
! cd squad/ && git pull

Already up to date.


In [3]:
! ls squad

args.py  environment.yml  LICENSE    README.md	setup.py  train.py
data	 layers.py	  models.py  save	test.py   util.py


In [4]:
! pip install ujson

     |████████████████████████████████| 194kB 5.0MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68031 sha256=e29e2adc102e9be0b82cffeebf13182c7ae5c855a5674f0e4bb887e2566a29ee
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson


In [5]:
! pip install transformers

     |████████████████████████████████| 368kB 4.9MB/s 
     |████████████████████████████████| 860kB 39.3MB/s 
     |████████████████████████████████| 645kB 39.6MB/s 
     |████████████████████████████████| 1.0MB 42.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=d8dcd3d4992a3851bd05208d0e8730c84934f8c61bbb5d9c72b9272fcc8ff1fe
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [6]:
! cd squad/ && python setup.py

glove.840B.300d.zip: 2.18GB [16:53, 2.15MB/s]                
Unzipping GloVe word vectors...
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
Pre-processing train examples...
100% 442/442 [01:51<00:00,  2.30it/s]
130319 questions in total
Pre-processing word vectors...
2196018it [03:30, 10422.42it/s]                 
88744 / 107454 tokens have corresponding word embedding vector
Pre-processing char vectors...
1374 tokens have corresponding char embedding vector
Pre-processing dev examples...
100% 16/16 [00:04<00:00,  3.47it/s]
6078 questions in total
Converting train examples to indices...
130319it [00:45, 2850.01it/s]
tcmalloc: large alloc 3326009344 bytes == 0x13900a000 @  0x7fcb0db6f1e7 0x7fcb0b64ef71 0x7fcb0b6b255d 0x7fcb0b6b5e28 0x7fcb0b6b63e5 0x7fcb0b74

In [0]:
import numpy.random as random
import numpy as np
seed = 224
random.seed(seed)
import torch
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [0]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import torch.utils.data as data

from squad.util import collate_fn, SQuAD
from collections import OrderedDict
from json import dumps
from tqdm import tqdm
from ujson import load as json_load
#import pytorch_lightning as pl

from squad import layers, util
import os

DIR = os.getcwd()

In [9]:
! pip install tensorboardX

     |████████████████████████████████| 194kB 4.8MB/s 


In [57]:
! cd squad && python test.py --split dev --name test1 --load_path a

[12.03.19 02:12:29] Args: {
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "hidden_size": 100,
    "load_path": "a",
    "load_paths": [
        "save/baseline0.tar",
        "save/baseline1.tar",
        "save/baseline2.tar",
        "save/baseline3.tar"
    ],
    "max_ans_len": 15,
    "name": "test1",
    "num_visuals": 10,
    "num_workers": 4,
    "save_dir": "./save/test/test1-26",
    "split": "dev",
    "sub_file": "submission.csv",
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[12.03.19 02:12:30] Loading embeddings...
[12.03.19 02:12:34] Building model...
[12.03.19 02:12:38] Building dataset...
[12.03.19 02:12:39] Evaluating on dev split...
100% 5951/5951 [00:21

##### F1 Score on Dev Set: 64.59
##### EM on Dev Set: 61.62